In [1]:
import pickle
import os
import pandas as pd
import multiprocessing as mp
from skimage.data import imread
from sklearn.ensemble import RandomForestClassifier

In [2]:
clf = pickle.load(open('/home/mashrin/input/camera_model.pkl', 'rb'))
test_path = '/home/mashrin/input/check'

In [3]:
test_images = []
for fname in sorted(os.listdir(test_path)):
    test_images.append(fname)
test = pd.DataFrame(test_images, columns=['fname'])

In [4]:
def color_stats(q, iolock):
    
    while True:
        
        img_path = q.get()
        if img_path is None:
            break
            
        if type(img_path) is tuple:
            img = imread(train_path/img_path[0]/img_path[1])
            key = img_path[1]
        else:
            img = imread('/home/mashrin/input/check/'+img_path)
            key = img_path

        if img.shape == (2,):
            img = img[0]

        color_info[key] = (img[:, :, 0].mean(), img[:, :, 1].mean(), img[:, :, 2].mean(),
                           img[:, :, 0].std(),  img[:, :, 1].std(),  img[:, :, 2].std())    

In [5]:
cols = ['a0', 'a1', 'a2', 's0', 's1', 's2']

for col in cols:
    test[col] = None

In [6]:
NCORE = 8

color_info = mp.Manager().dict()

q = mp.Queue(maxsize=NCORE)
iolock = mp.Lock()
pool = mp.Pool(NCORE, initializer=color_stats, initargs=(q, iolock))

for i in test_images:
    q.put(i)
    
for _ in range(NCORE):  
    q.put(None)
pool.close()
pool.join()

color_info = dict(color_info)

In [7]:
for n, col in enumerate(cols):
    test[col] = test['fname'].apply(lambda x: color_info[x][n])

In [8]:
X_test = test[cols].values

In [9]:
y_pred = clf.predict(X_test)
print(test_images)
print(y_pred)

['(iP4s)26.jpg']
['iPhone-4s']
